In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
import psycopg2

In [2]:
connection_string = psycopg2.connect(host="mypostgresdb.cnm9emj4w74o.us-east-2.rds.amazonaws.com", port = 5432, database="project3", user="rootlh", password="password")

In [3]:
pip install -U textblob


Requirement already up-to-date: textblob in /Users/hangli/opt/anaconda3/lib/python3.7/site-packages (0.15.3)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from textblob import TextBlob


In [5]:
# cur.execute("""SELECT * FROM posts_info""")
# query_results = cur.fetchall()
# query_results

In [6]:
df = pd.read_sql_query('select * from post_info', con=connection_string)
df.head()

,type,posts
0,INTJ,'https://www.youtube.com/watch?v=8IEQpfA528M ...
1,INFP,"'It's one of my daily 'music doodles', one min..."
2,INFP,'NO. But I have dyscalculia. :X|||I've learned...
3,ISTP,"'288970 It's been a while, guys. Also I bleach..."
4,INFP,'29 and still have it. Never had a girlfriend....


In [7]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
print(df.head())

   type                                              posts  words_per_comment
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86
1  INFP  'It's one of my daily 'music doodles', one min...              17.46
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90
4  INFP  '29 and still have it. Never had a girlfriend....              27.98


In [8]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
df['question_per_comment'] = df['posts'].apply(lambda x: x.count('?')/50)
df['excl_per_comment'] = df['posts'].apply(lambda x: x.count('!')/50)
df['upper_case'] = df['posts'].str.findall(r'[A-Z]').str.len()/50
# df['adj'] = df['posts'].str.findall(r'[A-Z]').str.len()/50
df[['polarity', 'subjectivity']] = df['posts'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
# df['emojis'] = df['posts'].str.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]').str.len()
df['ellipsis_per_comment'] = df['posts'].apply(lambda x: x.count('...')/50)


print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   
5  INFJ  'It gets to be pretty obvious the lifestyle di...              20.98   
6  INTP  'I used to find this difficult until I realize...              23.80   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...              33.88   
8  ENFP  'Can I just say I'm so glad and thankful that ...              26.28   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...              21.80   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                  0.68    

In [9]:
df

,type,posts,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,ellipsis_per_comment
0,INTJ,'https://www.youtube.com/watch?v=8IEQpfA528M ...,22.86,0.68,0.00,3.64,0.101422,0.546946,0.60
1,INFP,"'It's one of my daily 'music doodles', one min...",17.46,0.30,0.12,4.68,0.067913,0.542686,0.34
2,INFP,'NO. But I have dyscalculia. :X|||I've learned...,13.30,0.12,0.06,3.68,0.128141,0.579492,0.32
3,ISTP,"'288970 It's been a while, guys. Also I bleach...",10.90,0.16,0.26,2.50,0.052717,0.557769,0.10
4,INFP,'29 and still have it. Never had a girlfriend....,27.98,0.16,0.06,4.42,0.046999,0.505684,0.80
...,...,...,...,...,...,...,...,...,...
8670,INTJ,'What about when you hurt someone else? Do yo...,16.34,0.20,0.02,4.42,0.181248,0.573210,0.50
8671,INFJ,'Any INFJ having a relationship with a ENFP? B...,29.80,0.34,0.28,6.32,0.189571,0.557296,0.90
8672,INFJ,'That's very interesting how you're in a relat...,29.10,0.02,0.02,4.52,0.189957,0.518341,0.74
8673,ESTP,'I don't feel bad at all about that and I don'...,23.74,0.46,0.30,5.18,0.151486,0.527806,0.72


In [10]:
map1 = {"I": 0, "E": 1}
map2 = {"N": 0, "S": 1}
map3 = {"T": 0, "F": 1}
map4 = {"J": 0, "P": 1}
df['I-E'] = df['type'].astype(str).str[0]
df['I-E'] = df['I-E'].map(map1)
df['N-S'] = df['type'].astype(str).str[1]
df['N-S'] = df['N-S'].map(map2)
df['T-F'] = df['type'].astype(str).str[2]
df['T-F'] = df['T-F'].map(map3)
df['J-P'] = df['type'].astype(str).str[3]
df['J-P'] = df['J-P'].map(map4)
print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   
5  INFJ  'It gets to be pretty obvious the lifestyle di...              20.98   
6  INTP  'I used to find this difficult until I realize...              23.80   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...              33.88   
8  ENFP  'Can I just say I'm so glad and thankful that ...              26.28   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...              21.80   

   question_per_comment  excl_per_comment  upper_case  polarity  subjectivity  \
0                  0.68    

In [11]:
X = df.drop(['type','posts','I-E','N-S','T-F','J-P'], axis=1)
print(X.shape)

(8675, 7)


In [12]:
X.shape

(8675, 7)

In [13]:
df['type'].unique()

array(['INTJ', 'INFP', 'ISTP', 'INFJ', 'INTP', 'ENFP', 'ESFP', 'ENTP',
       'ENFJ', 'ESTP', 'ISFP', 'ENTJ', 'ISTJ', 'ESTJ', 'ISFJ', 'ESFJ'],
      dtype=object)

In [14]:
# XX = df.drop(['type','posts','I-E'], axis=1).values
# yy = df['I-E'].values

# print(yy.shape)
# print(XX.shape)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [16]:
# Introvert vs. Extrovert
y = df['I-E']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)


In [17]:
X_train

,words_per_comment,question_per_comment,excl_per_comment,upper_case,polarity,subjectivity,ellipsis_per_comment
5495,25.74,0.28,0.02,4.50,0.193355,0.532972,0.62
7205,30.82,0.18,0.36,6.76,0.206285,0.601249,1.16
3889,30.08,0.08,0.08,5.74,0.101339,0.516682,0.72
94,13.90,0.28,0.14,2.40,0.077992,0.690285,0.20
6675,17.58,0.12,0.02,2.88,0.117476,0.608326,0.24
...,...,...,...,...,...,...,...
3704,21.56,0.26,0.36,5.80,0.095862,0.523073,0.54
163,27.06,0.26,0.22,5.92,0.081877,0.531350,0.72
6012,31.16,0.32,0.02,6.06,0.030651,0.505661,0.74
6558,24.32,0.18,0.00,7.22,0.106238,0.503160,0.60


In [18]:
y

0       0
1       0
2       0
3       0
4       0
       ..
8670    0
8671    0
8672    0
8673    1
8674    0
Name: I-E, Length: 8675, dtype: int64

In [19]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
X_train_scaled

array([[0.6908993 , 0.11570248, 0.005     , ..., 0.73143713, 0.5329717 ,
        0.18674699],
       [0.82767905, 0.07438017, 0.09      , ..., 0.7409577 , 0.6012491 ,
        0.34939759],
       [0.80775444, 0.03305785, 0.02      , ..., 0.66368255, 0.51668164,
        0.21686747],
       ...,
       [0.8368336 , 0.1322314 , 0.005     , ..., 0.61163283, 0.50566111,
        0.22289157],
       [0.65266559, 0.07438017, 0.        , ..., 0.66728972, 0.50315976,
        0.18072289],
       [0.63004847, 0.1322314 , 0.04      , ..., 0.69210999, 0.53853771,
        0.21084337]])

In [21]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [22]:
encoded_y_train

array([0, 0, 0, ..., 0, 1, 0])

In [23]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [24]:
y_train_categorical

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [26]:
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=7))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='relu'))

In [27]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1600      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
204/204 - 0s - loss: 0.5437 - accuracy: 0.7658
Epoch 2/60
204/204 - 0s - loss: 0.5368 - accuracy: 0.7687
Epoch 3/60
204/204 - 0s - loss: 0.5318 - accuracy: 0.7699
Epoch 4/60
204/204 - 0s - loss: 0.5298 - accuracy: 0.7704
Epoch 5/60
204/204 - 0s - loss: 0.5284 - accuracy: 0.7698
Epoch 6/60
204/204 - 0s - loss: 0.5288 - accuracy: 0.7699
Epoch 7/60
204/204 - 0s - loss: 0.5328 - accuracy: 0.7701
Epoch 8/60
204/204 - 0s - loss: 0.5309 - accuracy: 0.7691
Epoch 9/60
204/204 - 0s - loss: 0.5288 - accuracy: 0.7694
Epoch 10/60
204/204 - 0s - loss: 0.5289 - accuracy: 0.7696
Epoch 11/60
204/204 - 0s - loss: 0.5274 - accuracy: 0.7690
Epoch 12/60
204/204 - 0s - loss: 0.5303 - accuracy: 0.7701
Epoch 13/60
204/204 - 0s - loss: 0.5261 - accuracy: 0.7698
Epoch 14/60
204/204 - 0s - loss: 0.5263 - accuracy: 0.7688
Epoch 15/60
204/204 - 0s - loss: 0.5268 - accuracy: 0.7687
Epoch 16/60
204/204 - 0s - loss: 0.5255 - accuracy: 0.7694
Epoch 17/60
204/204 - 0s - loss: 0.5243 - accuracy: 0.7688
Epoch 

In [30]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

68/68 - 0s - loss: nan - accuracy: 0.7722
Normal Neural Network - Loss: nan, Accuracy: 0.7722452878952026


In [31]:
#Intuitive vs. Sensing
y = df['N-S'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

In [32]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [34]:
model2 = Sequential()
model2.add(Dense(units=200, activation='relu', input_dim=7))
model2.add(Dense(units=200, activation='relu'))
model2.add(Dense(units=2, activation='relu'))

In [35]:
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
model2.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               1600      
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402       
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________


In [37]:
model2.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
204/204 - 0s - loss: nan - accuracy: 0.8624
Epoch 2/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 3/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 4/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 5/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 6/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 7/60
204/204 - 0s - loss: nan - accuracy: 0.8624
Epoch 8/60
204/204 - 0s - loss: nan - accuracy: 0.8624
Epoch 9/60
204/204 - 0s - loss: nan - accuracy: 0.8624
Epoch 10/60
204/204 - 0s - loss: nan - accuracy: 0.8624
Epoch 11/60
204/204 - 0s - loss: nan - accuracy: 0.8624
Epoch 12/60
204/204 - 0s - loss: nan - accuracy: 0.8624
Epoch 13/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 14/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 15/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 16/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 17/60
204/204 - 1s - loss: nan - accuracy: 0.8624
Epoch 18/60
204/204 - 1s - loss: nan - accuracy: 0.8624
E

In [38]:
model_loss, model_accuracy = model2.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

68/68 - 0s - loss: nan - accuracy: 0.8608
Normal Neural Network - Loss: nan, Accuracy: 0.8607653379440308


In [39]:
# Thinking vs. Feeling
y = df['T-F'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)


In [40]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [42]:
model3 = Sequential()
model3.add(Dense(units=200, activation='sigmoid', input_dim=7))
model3.add(Dense(units=200, activation='sigmoid'))
model3.add(Dense(units=2, activation='sigmoid'))

In [43]:
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [44]:
model3.summary() 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 200)               1600      
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 402       
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________


In [45]:
model3.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
204/204 - 0s - loss: 0.6925 - accuracy: 0.5400
Epoch 2/60
204/204 - 0s - loss: 0.6902 - accuracy: 0.5418
Epoch 3/60
204/204 - 0s - loss: 0.6898 - accuracy: 0.5418
Epoch 4/60
204/204 - 0s - loss: 0.6894 - accuracy: 0.5418
Epoch 5/60
204/204 - 1s - loss: 0.6893 - accuracy: 0.5418
Epoch 6/60
204/204 - 0s - loss: 0.6879 - accuracy: 0.5418
Epoch 7/60
204/204 - 0s - loss: 0.6850 - accuracy: 0.5453
Epoch 8/60
204/204 - 1s - loss: 0.6828 - accuracy: 0.5427
Epoch 9/60
204/204 - 1s - loss: 0.6712 - accuracy: 0.5629
Epoch 10/60
204/204 - 0s - loss: 0.6651 - accuracy: 0.5882
Epoch 11/60
204/204 - 1s - loss: 0.6636 - accuracy: 0.5921
Epoch 12/60
204/204 - 1s - loss: 0.6632 - accuracy: 0.5899
Epoch 13/60
204/204 - 1s - loss: 0.6610 - accuracy: 0.6005
Epoch 14/60
204/204 - 1s - loss: 0.6619 - accuracy: 0.5931
Epoch 15/60
204/204 - 0s - loss: 0.6623 - accuracy: 0.5956
Epoch 16/60
204/204 - 0s - loss: 0.6596 - accuracy: 0.5988
Epoch 17/60
204/204 - 1s - loss: 0.6583 - accuracy: 0.6041
Epoch 

In [46]:
model_loss, model_accuracy = model3.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

68/68 - 0s - loss: 0.6364 - accuracy: 0.6445
Normal Neural Network - Loss: 0.6363771557807922, Accuracy: 0.6445366740226746


In [47]:
# Judging vs. Perceiving
y = df['J-P'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)


In [48]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [49]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [50]:
model4 = Sequential()
model4.add(Dense(units=200, activation='sigmoid', input_dim=7))
model4.add(Dense(units=200, activation='sigmoid'))
model4.add(Dense(units=2, activation='sigmoid'))

In [51]:
model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [52]:
model4.summary() 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 200)               1600      
_________________________________________________________________
dense_10 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 402       
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________


In [53]:
model4.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
204/204 - 1s - loss: 0.6740 - accuracy: 0.6027
Epoch 2/60
204/204 - 1s - loss: 0.6746 - accuracy: 0.6030
Epoch 3/60
204/204 - 0s - loss: 0.6743 - accuracy: 0.6030
Epoch 4/60
204/204 - 1s - loss: 0.6742 - accuracy: 0.6030
Epoch 5/60
204/204 - 0s - loss: 0.6731 - accuracy: 0.6030
Epoch 6/60
204/204 - 0s - loss: 0.6744 - accuracy: 0.6030
Epoch 7/60
204/204 - 1s - loss: 0.6730 - accuracy: 0.6030
Epoch 8/60
204/204 - 1s - loss: 0.6727 - accuracy: 0.6030
Epoch 9/60
204/204 - 1s - loss: 0.6735 - accuracy: 0.6030
Epoch 10/60
204/204 - 1s - loss: 0.6726 - accuracy: 0.6030
Epoch 11/60
204/204 - 0s - loss: 0.6725 - accuracy: 0.6030
Epoch 12/60
204/204 - 0s - loss: 0.6726 - accuracy: 0.6030
Epoch 13/60
204/204 - 1s - loss: 0.6722 - accuracy: 0.6030
Epoch 14/60
204/204 - 1s - loss: 0.6718 - accuracy: 0.6030
Epoch 15/60
204/204 - 0s - loss: 0.6732 - accuracy: 0.6030
Epoch 16/60
204/204 - 1s - loss: 0.6721 - accuracy: 0.6030
Epoch 17/60
204/204 - 1s - loss: 0.6722 - accuracy: 0.6030
Epoch 

In [54]:
model_loss, model_accuracy = model4.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

68/68 - 0s - loss: 0.6684 - accuracy: 0.6077
Normal Neural Network - Loss: 0.6683854460716248, Accuracy: 0.6076533198356628


In [55]:
# Overall
y = df['type'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45)

In [56]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [57]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [58]:
model5 = Sequential()
model5.add(Dense(units=200, activation='sigmoid', input_dim=7))
model5.add(Dense(units=200, activation='sigmoid'))
model5.add(Dense(units=16, activation='sigmoid'))

In [59]:
model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
model5.summary() 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 200)               1600      
_________________________________________________________________
dense_13 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_14 (Dense)             (None, 16)                3216      
Total params: 45,016
Trainable params: 45,016
Non-trainable params: 0
_________________________________________________________________


In [61]:
model5.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
204/204 - 1s - loss: 2.3127 - accuracy: 0.2049
Epoch 2/60
204/204 - 1s - loss: 2.2924 - accuracy: 0.2073
Epoch 3/60
204/204 - 1s - loss: 2.2901 - accuracy: 0.2073
Epoch 4/60
204/204 - 1s - loss: 2.2889 - accuracy: 0.2073
Epoch 5/60
204/204 - 1s - loss: 2.2880 - accuracy: 0.2073
Epoch 6/60
204/204 - 1s - loss: 2.2870 - accuracy: 0.2073
Epoch 7/60
204/204 - 1s - loss: 2.2860 - accuracy: 0.2073
Epoch 8/60
204/204 - 1s - loss: 2.2841 - accuracy: 0.2073
Epoch 9/60
204/204 - 1s - loss: 2.2829 - accuracy: 0.2073
Epoch 10/60
204/204 - 1s - loss: 2.2805 - accuracy: 0.2073
Epoch 11/60
204/204 - 1s - loss: 2.2789 - accuracy: 0.2073
Epoch 12/60
204/204 - 1s - loss: 2.2737 - accuracy: 0.2073
Epoch 13/60
204/204 - 1s - loss: 2.2665 - accuracy: 0.2073
Epoch 14/60
204/204 - 1s - loss: 2.2568 - accuracy: 0.2073
Epoch 15/60
204/204 - 1s - loss: 2.2497 - accuracy: 0.2080
Epoch 16/60
204/204 - 1s - loss: 2.2447 - accuracy: 0.2077
Epoch 17/60
204/204 - 1s - loss: 2.2421 - accuracy: 0.2100
Epoch 

In [62]:
model_loss, model_accuracy = model5.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

68/68 - 0s - loss: 2.2135 - accuracy: 0.2411
Normal Neural Network - Loss: 2.2134501934051514, Accuracy: 0.2411249428987503
